In [1]:
import scrapy
import json
import requests
import requests_cache
import numpy as np
import pandas as pd
import seaborn as sns
import urlparse
import csv
import os
import os.path
from bs4 import BeautifulSoup
import sys
from lxml import html
import lxml.html as lx

requests_cache.install_cache('demo_cache')

In [2]:
def box_search(year):
    
    base_url = 'http://www.boxofficemojo.com/yearly/chart/?yr='
    url = base_url + str(year)
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'lxml')
 
    res_2 = soup.find_all('td')
    names = []
    box = []
    studio = []
    for i in range(0,100):
        
        names.append(res_2[21+9*i].find_all('b')[0].text)
        box.append(res_2[23+9*i].find_all('b')[0].text)
        studio.append(res_2[22+9*i].find_all('a')[0].text)
        
        params = {'year':year, 'name':names, 'box':box,}
    
    return pd.DataFrame(params)

In [3]:
def popular_movie(year,pages,num):
    count = 0
    names = []
    
    for i in range(1,(pages+1)):
        url = "http://www.imdb.com/search/title?sort=moviemeter,asc&title_type=feature&year=" + str(year) + "," +str(year) + "&" + "page=" + str(i) + "&" + "ref_=adv_prv"        
        r = requests.get(url)
        soup = BeautifulSoup(r.content, "html.parser")
        search_results = soup.find_all("p", class_="sort-num_votes-visible")
        vote = int(search_results[0].find_all('span')[1].text.replace(',',''))
        soup = BeautifulSoup(r.content, "html.parser")
        search_results = soup.find_all("div", class_="lister-item-content")
        

        for result in search_results:
            try:
                vote = int(result.find_all('p')[3].find_all('span')[1].text.replace(',',''))
                if vote < 10000:
                    continue
            except:
                continue
            header = result.find("h3", class_="lister-item-header")
            title = header.a
            b = title.get_text().encode('utf-8')
            names.append(b)
            count = count + 1
            if count == num:
                return names
            
    return names

In [4]:
def movie_search(term):
    
    url = 'http://www.omdbapi.com/'
    
    params = {'t':term}
    a_final = "&".join("{}={}".format(a,b) for a,b in params.items())
    url_final = url + '?' + a_final
    
    response = requests.get(url_final)
    js = response.json()
    
    return js

In [5]:
def rating_ratio(movie):
    
    imdb = movie_search(movie).values()[-1]
    url = 'http://www.imdb.com/title/' + str(imdb) + '/ratings?ref_=tt_ov_rt'
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "html.parser")
    res_2 = soup.find_all('td')
    
    rates = []
    for i in xrange(14):
        string = res_2[38 + 3*i].find_all('img')[0].text
        string = string.replace(u'\xa0', u'')
        string  = str(string)
        rates.append(string)
        
    params = {'males':rates[0], 'females':rates[1],'<18':rates[2], 'males<18':rates[3],'females<18':rates[4], '18-29':rates[5],'males 18-29':rates[6], 'females 18-29':rates[7],'30-44':rates[8],'males 30-44':rates[9], 'females 30-44':rates[10], '45+':rates[11],'males 45+':rates[12], 'females 45+':rates[13]}
    return params

In [6]:
rating_ratio('Sing')

{'18-29': '7.2',
 '30-44': '7.1',
 '45+': '7.2',
 '<18': '7.7',
 'females': '7.5',
 'females 18-29': '7.5',
 'females 30-44': '7.5',
 'females 45+': '7.7',
 'females<18': '8.0',
 'males': '7.1',
 'males 18-29': '7.1',
 'males 30-44': '7.1',
 'males 45+': '7.2',
 'males<18': '7.5'}